In [1]:
import numpy 
from sklearn.metrics import jaccard_score
from collections import defaultdict
import json
import random

In [16]:
import random

In [4]:
data_path = '../data/cleaned/RC_2023-01_2.json'
with open(data_path, 'r') as fh:
    comments = json.load(fh)

### Simple Data Stats

In [7]:
print(f'No. of comments in the data - {len(comments)}')

No. of comments in the data - 407380


In [5]:
n_subreddit = len(set([comment['subreddit_id'] for comment in comments]))
print(f'No. of unique subreddits : {n_subreddit}')

No. of unique subreddits : 6217


In [6]:
n_users = len(set([comment['author'] for comment in comments]))
print(f'No. of unique users : {n_users}')

No. of unique users : 194815


### Data Transformation

In [14]:
usersperitem = defaultdict(set)
itemsperuser = defaultdict(set)
item_name = defaultdict()
for comment in comments:
    user = comment['author_fullname']
    item = comment['subreddit_id']
    item_name[item] = comment['subreddit']
    usersperitem[item].add(user)
    itemsperuser[user].add(item)

In [57]:
users = [u for u in itemsperuser]

### Jaccard Similarity

In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [20]:
# Retrieve the most similar item based on jaccard similarity
def mostSimilar(i, N):
    similarities = []
    users = usersperitem[i]
    for i2 in usersperitem:
        if i2 == i: continue
        sim = Jaccard(users, usersperitem[i2])
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:N]

In [140]:
idx = random.randint(0, len(comments))
query = comments[idx]['subreddit_id']
similar_items = mostSimilar(query, 5)
# print(similar_items)
print(item_name[query])
for sim, _id in similar_items:
    print(sim, item_name[_id])

WoWs_Legends
0.017857142857142856 patientgamers
0.0021551724137931034 wholesomememes
0.000657030223390276 memes
0.0 ShitpostBR
0.0 CountryMusicStuff


In [68]:
## Given a user find the subreddits to reccommend

# 2: Jaccard similarity with most similar user who has consumed i
def score(u, i, userHistory):
    bestsim = None
    for v in usersperitem[i]:
        if u == v:
            continue
        sim = Jaccard(userHistory, itemsperuser[v])
        if bestsim == None or sim > bestsim:
            bestsim = sim
    if bestsim == None:
        return 0
    return bestsim

def rec(u):
    history = itemsperuser[u]
    bestitem = None
    bestscore = None
    for i in item_name:
        if i in history:
            continue
        s = score(u, i, history)
        if bestitem == None or s > bestscore:
            bestitem = i
            bestscore = s
    return bestitem, bestscore

In [135]:
while True:
    u = random.sample(users, 1)[0]
    if len(itemsperuser[u]) > 3:
        break
item, similarity_score = rec(u)
print(f'recommended - {item_name[item]}')
print('user history')
for item in itemsperuser[u]:
    print(item_name[item])

recommended - wallstreetbets
user history
AndrewTateTalk
relationship_advice
Watches
NorthCarolina


## Evaluating the model

In [ ]:
## Train Test Split 